In [17]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, ttest_rel, wilcoxon

In [18]:
model1 = 'DenseNet169'
model2 = 'SimpleCNN'
model3 = 'EnhancedCNN'

batch_size = 10
epochs = 50
k = 5


In [19]:
model1_results_approach1 = pd.read_csv(f'results/{model1}_epoch{epochs}_batch{batch_size}_val_results.txt', delimiter=', ')
model2_results_approach1 = pd.read_csv(f'results/{model2}_epoch{epochs}_batch{batch_size}_val_results.txt', delimiter=', ')
model3_results_approach1 = pd.read_csv(f'results/{model3}_epoch{epochs}_batch{batch_size}_val_results.txt', delimiter=', ')

# Approach 2 - k-fold cross-validation
model1_results_approach2 = pd.read_csv(f'results/{model1}_k{k}_epoch{epochs}_batch{batch_size}_val_results.txt', delimiter=', ')
model2_results_approach2 = pd.read_csv(f'results/{model2}_k{k}_epoch{epochs}_batch{batch_size}_val_results.txt', delimiter=', ')
model3_results_approach2 = pd.read_csv(f'results/{model3}_k{k}_epoch{epochs}_batch{batch_size}_val_results.txt', delimiter=', ')


C:\Users\PC\AppData\Local\Temp\ipykernel_7996\3803886145.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  model1_results_approach1 = pd.read_csv(f'results/{model1}_epoch{epochs}_batch{batch_size}_val_results.txt', delimiter=', ')
C:\Users\PC\AppData\Local\Temp\ipykernel_7996\3803886145.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  model2_results_approach1 = pd.read_csv(f'results/{model2}_epoch{epochs}_batch{batch_size}_val_results.txt', delimiter=', ')
C:\Users\PC\AppData\Local\Temp\ipykernel_7996\3803886145.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does no

In [20]:
model1_results_approach2_avg = model1_results_approach2.groupby('Epoch').mean()
model2_results_approach2_avg = model2_results_approach2.groupby('Epoch').mean()
model3_results_approach2_avg = model3_results_approach2.groupby('Epoch').mean()


In [21]:
metrics = ['F1 Score', 'Val Acc', 'AUC']

In [22]:
for metric in metrics:
    print(f"Metric: {metric}")
    results_model1_approach1 = model1_results_approach1[metric]
    results_model1_approach2 = model1_results_approach2_avg[metric]

    results_model2_approach1 = model2_results_approach1[metric]
    results_model2_approach2 = model2_results_approach2_avg[metric]

    results_model3_approach1 = model3_results_approach1[metric]
    results_model3_approach2 = model3_results_approach2_avg[metric]

    # Perform the paired tests
    print("Approach 1 vs Approach 2:")
    is_normal = True

    for results_model_approach1, results_model_approach2, model_name in [(results_model1_approach1, results_model1_approach2, model1),
                                                             (results_model2_approach1, results_model2_approach2, model2),
                                                             (results_model3_approach1, results_model3_approach2, model3)]:

        _, p_value = shapiro(results_model_approach1 - results_model_approach2)
        print(f"Tests for model:{model_name}")
        print("Shapiro-Wilk p-value:", p_value)

        if p_value < 0.05:
            is_normal = False

        if is_normal:
            # Perform t-test paired for normal distribution
            t_statistic, p_value_ttest = ttest_rel(results_model1_approach1, results_model1_approach2)
            print("T-Test p-values:", p_value_ttest)

            if np.asarray(p_value_ttest) < 0.05:
                print("There is a significant difference between the two approaches (p-value < 0.05)")
            else:
                print("There is no significant difference between the two approaches (p-value >= 0.05)")
        else:
            # Perform Wilcoxon test for non-normal distribution
            _, p_value_wilcoxon = wilcoxon(results_model1_approach1, results_model1_approach2)
            print("Wilcoxon p-values:", p_value_wilcoxon)

            if np.asarray(p_value_wilcoxon) < 0.05:
                print("There is a significant difference between the two approaches (p-value < 0.05)")
            else:
                print("There is no significant difference between the two approaches (p-value >= 0.05)")

Metric: F1 Score
Approach 1 vs Approach 2:
Tests for model:DenseNet169
Shapiro-Wilk p-value: 1.0
T-Test p-values: 1.297485963491924e-40
There is a significant difference between the two approaches (p-value < 0.05)
Tests for model:SimpleCNN
Shapiro-Wilk p-value: 1.0
T-Test p-values: 1.297485963491924e-40
There is a significant difference between the two approaches (p-value < 0.05)
Tests for model:EnhancedCNN
Shapiro-Wilk p-value: 1.0
T-Test p-values: 1.297485963491924e-40
There is a significant difference between the two approaches (p-value < 0.05)
Metric: Val Acc
Approach 1 vs Approach 2:
Tests for model:DenseNet169
Shapiro-Wilk p-value: 1.0
T-Test p-values: 1.6953800898590885e-26
There is a significant difference between the two approaches (p-value < 0.05)
Tests for model:SimpleCNN
Shapiro-Wilk p-value: 1.0
T-Test p-values: 1.6953800898590885e-26
There is a significant difference between the two approaches (p-value < 0.05)
Tests for model:EnhancedCNN
Shapiro-Wilk p-value: 1.0
T-Test p